In [1]:
%matplotlib inline

import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dataPath = '../PedestrianData/IntentionData'
pd.__version__

'0.24.2'

# Curating final set of possible features

In [23]:
files = os.listdir(dataPath)

features = np.empty((len(files), 32), dtype=list)

for i in range(len(files)):
    file = dataPath + "/" + files[i]
    df = pd.read_csv(file, sep=",", header=None, 
                 names=["Time", 
                        "Pos_x", "Pos_y", "Pos_z", 
                        "Gaz_x", "Gaz_y", "Gaz_z", 
                        "Vel_x", "Vel_z",
                        "Acc_x", "Acc_z",
                        "Vel_Mag", "Acc_Mag",
                        "Vel_Ang", "Acc_Ang", "Gaz_Ang",
                        "Vel_Bin", "Acc_Bin", "Gaz_Bin", "Chg_Bin"          
                       ])
   
    
    # Replacing NaN values with -1 for bins and 0 for magnitudes
    df["Vel_Bin"].fillna(-1, inplace=True)
    df["Acc_Bin"].fillna(-1, inplace=True)
    
    features[i, 0] = df["Pos_x"]
    features[i, 1] = df["Pos_z"]
    features[i, 2] = df["Vel_Mag"]
    features[i, 3] = df["Acc_Mag"]
    
    # Filling the rest of the features with each direction bin
    vel_bins = np.histogram(np.array(df["Vel_Bin"]), bins=[1, 2, 3, 4, 5, 6, 7, 8], density=False)[0]
    acc_bins = np.histogram(np.array(df["Acc_Bin"]), bins=[1, 2, 3, 4, 5, 6, 7, 8], density=False)[0]
    gaze_bins = np.histogram(np.array(df["Gaz_Bin"]), bins=[1, 2, 3, 4, 5, 6, 7, 8], density=False)[0]
    change_bins = np.histogram(np.array(df["Chg_Bin"]), bins=[1, 2, 3, 4, 5, 6, 7, 8], density=False)[0]
    for j in range(len(gaze_bins)):
        features[i, j+3] = gaze_bins[j]
    for k in range(len(vel_bins)):
        features[i, k+j+3] = vel_bins[k]
    for l in range(len(vel_bins)):
        features[i, l+k+j+3] = vel_bins[l]
    for o in range(len(acc_bins)):
        features[i, o+l+k+j+3] = acc_bins[o]
        
# Removing data with crossing time of '0'
features = features[features[:, 0] != 0]

C:\Users\Rafael\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
